# Voxel-Based Analysis with SVM

This notebook demonstrates a voxel-based approach using Support Vector Machines (SVM). This is useful when you want fine-grained spatial resolution beyond atlas parcellations.

## Approach Overview

**Feature Extraction**: Voxel-based (statistically selected)  
**Feature Selection**: VoxelSelectorFromImages (univariate selection)  
**Model**: Support Vector Machine with RBF kernel  
**Validation**: Leave-One-Out Cross-Validation

## When to Use This

- **Small, focal effects**: When TES effects are localized to specific voxels rather than entire regions
- **Exploratory analysis**: To identify optimal stimulation sites without atlas constraints
- **High spatial precision**: When millimeter-level accuracy matters

## Cautions

- **Curse of dimensionality**: Voxel features = ~50,000-200,000 dimensions
- **Overfitting risk**: Must use aggressive feature selection
- **Computational cost**: SVM training is O(n²) to O(n³)

In [ ]:
import teslearn as tl
from teslearn.data import load_dataset_from_csv, NiftiLoader
from teslearn.features import VoxelFeatureExtractor
from teslearn.models import SVMModel
from teslearn.selection import VoxelSelectorFromImages
from teslearn.cv import LeaveOneOutValidator, StratifiedKFoldValidator

import numpy as np
import matplotlib.pyplot as plt

## 1. Data Loading

**Best Practice**: For voxel-based analysis, ensure all images are in the same space (MNI) and have the same dimensions.

In [ ]:
# Load dataset
dataset = load_dataset_from_csv(
    csv_path='data/subjects.csv',
    target_col='response',
    task='classification'
)

loader = NiftiLoader()
images, indices = loader.load_dataset_images(dataset)
y = dataset.get_targets()

print(f"Dataset: {len(images)} subjects")
print(f"Image shape: {images[0].shape}")

## 2. Voxel Selection Strategy

**Critical Step**: We cannot use all voxels (curse of dimensionality). Instead:

1. **Univariate selection**: Select voxels showing group differences
2. **Spatial clustering**: Keep only significant clusters (optional)
3. **Conservative threshold**: p < 0.001 or stricter

**Why this matters**: Including non-informative voxels adds noise and increases overfitting.

In [ ]:
# Select voxels based on univariate statistical test
voxel_selector = VoxelSelectorFromImages(
    p_threshold=0.001,      # Conservative p-value
    test_type='ttest',      # T-test for binary classification
    correction=None         # Can use 'bonferroni' for strict correction
)

# Fit selector on training data
voxel_selector.fit(images, y)

# Get selected coordinates
selected_coords = voxel_selector.get_voxel_coordinates()
print(f"Selected {len(selected_coords)} voxels from {np.prod(images[0].shape)} total")
print(f"Reduction: {100 * (1 - len(selected_coords)/np.prod(images[0].shape)):.1f}%")

# Create extractor for selected voxels
extractor = voxel_selector.create_voxel_extractor()

## 3. Visualize Selected Voxels

Visualize the spatial distribution of selected voxels to understand where predictive information lies.

In [ ]:
# Create a mask of selected voxels
mask = np.zeros(images[0].shape, dtype=bool)
for x, y_coord, z in selected_coords:
    mask[x, y_coord, z] = True

# Plot distribution across slices
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

sample_img = images[0]

for i, ax in enumerate(axes):
    z_slice = 20 + i * 10
    if z_slice < sample_img.shape[2]:
        ax.imshow(sample_img[:, :, z_slice], cmap='gray', alpha=0.5)
        ax.imshow(mask[:, :, z_slice], cmap='Reds', alpha=0.7)
        ax.set_title(f'Z={z_slice}')
        ax.axis('off')

plt.suptitle('Selected Voxels (Red) Overlay on Sample Image')
plt.tight_layout()
plt.show()

## 4. SVM Configuration

**Support Vector Machines** are well-suited for high-dimensional data:

- **Kernel trick**: RBF kernel handles non-linear relationships
- **Margin maximization**: Naturally resistant to overfitting
- **Works in high-D**: Effective even when features > samples

**Hyperparameters**:
- `C`: Regularization (smaller = stronger regularization)
- `gamma`: Kernel coefficient ('scale' is usually good)
- `probability=True`: Enable probability estimates (needed for ROC)

In [ ]:
# Configure SVM
model = SVMModel(
    kernel='rbf',           # Radial basis function kernel
    C=1.0,                  # Regularization parameter
    gamma='scale',          # Kernel coefficient
    class_weight='balanced', # Handle class imbalance
    probability=True,       # Enable probability estimates
    random_state=42
)

print("SVM Configuration:")
print(f"  Kernel: RBF")
print(f"  C: {model.C}")
print(f"  Gamma: {model.gamma}")

## 5. Leave-One-Out Cross-Validation

**When to use LOO**:
- Very small sample sizes (n < 30)
- When every subject is precious
- Unbiased but high variance estimates

**Trade-off**: LOO gives nearly unbiased estimates but has higher variance than k-fold.

In [ ]:
# Configure cross-validation
outer_cv = LeaveOneOutValidator()  # One subject left out for testing
inner_cv = StratifiedKFoldValidator(n_splits=3)  # 3-fold for feature selection

# Build pipeline
from teslearn.pipeline import TESPipeline

pipeline = TESPipeline(
    feature_extractor=extractor,
    feature_selector=None,  # Already selected voxels
    model=model,
    use_scaling=True        # Critical for SVM
)

# Perform cross-validation
result = tl.cross_validate(
    pipeline=pipeline,
    images=images,
    y=y,
    outer_validator=outer_cv,
    inner_validator=inner_cv
)

print(result.get_summary())

## 6. Interpret Voxel Importance

SVMs don't provide direct feature importance like logistic regression. However, we can:
1. Use permutation importance
2. Analyze support vectors
3. Map decision boundary in voxel space

In [ ]:
from teslearn.viz import create_stat_map

# Extract features for visualization
X = extractor.fit_transform(images)

# Fit model on all data for interpretation
pipeline.fit(images, y)

# Get predictions and decision values
decision_values = pipeline.model._model.decision_function(X)

print(f"Decision value range: [{decision_values.min():.2f}, {decision_values.max():.2f}]")
print(f"Mean absolute decision value: {np.abs(decision_values).mean():.2f}")

# Analyze class separation
responders = decision_values[y == 1]
non_responders = decision_values[y == 0]

plt.figure(figsize=(10, 6))
plt.hist(non_responders, bins=15, alpha=0.7, label='Non-responders', color='red')
plt.hist(responders, bins=15, alpha=0.7, label='Responders', color='blue')
plt.axvline(x=0, color='black', linestyle='--', label='Decision boundary')
plt.xlabel('SVM Decision Value')
plt.ylabel('Count')
plt.title('Class Separation in Decision Space')
plt.legend()
plt.show()

## Key Takeaways

1. **Voxel selection is critical**: Never use raw voxels; always select based on univariate statistics
2. **SVM works well in high-D**: RBF kernel handles non-linear relationships in voxel space
3. **Scaling is essential**: Always standardize features for SVM
4. **LOO for small N**: Use when sample size is very limited

## Comparison to Atlas Approach

| Aspect | Atlas-Based | Voxel-Based |
|--------|-------------|-------------|
| Resolution | Regional | Sub-regional |
| Interpretability | High (anatomical) | Lower (coordinate space) |
| Overfitting risk | Low | Higher |
| Sample size | Flexible | Requires larger N |
| Computational cost | Low | Higher |

## Best Practices

- Always validate voxel selection within CV loops
- Use conservative p-thresholds (0.001 or Bonferroni)
- Consider spatial smoothing before voxel selection
- Report number of selected voxels